In [1]:
from alignconv.converters.align_converter import Converter
import torchvision
import torch
from torch import nn
import copy
from alignconv.convs import TSMConv, AlignShiftConv
# m is a standard pytorch model
m1 = torchvision.models.resnet18()
# m1 = m1.features
# m1 = nn.DataParallel(m1)
# after convert, m is using AlignConv
# conv_cfg = dict(conv_type=TSMConv,padding_mode='zeros', 
#                  n_fold=8, align=True, inplace=True,
#                  ref_spacing=0.2, shift_padding_zero=True)
# wrapper = AlignWrapper(conv_cfg)
# m2 = wrapper(copy.deepcopy(m1))

In [2]:
tsm_conv_cfg = dict(conv_type=TSMConv, bias=True, padding_mode='zeros', 
                 n_fold=8, tsm=True, inplace=True,
                 shift_padding_zero=True)
alignshift_conv_cfg = dict(conv_type=AlignShiftConv, n_fold=8, alignshift=True, inplace=True,
                 ref_spacing=0.2, shift_padding_zero=True)

In [3]:
m2 = Converter(copy.deepcopy(m1), tsm_conv_cfg)

In [4]:
l = m2.state_dict()

In [5]:
m2.load_state_dict(l)

<All keys matched successfully>

In [6]:
batch_size = 4
in_channels = 1
d = 7
height = 320
width = 320
input = torch.rand(batch_size, in_channels, d, height, width).cuda()
m2.eval()
m2.cuda()
out = m2(input)
print(out)
m3 = nn.DataParallel(m2)
m3(input).sum()

tensor([[-0.0868, -0.2138, -0.2144,  ..., -0.1045, -0.3122, -0.0217],
        [-0.0857, -0.2163, -0.2122,  ..., -0.1077, -0.3134, -0.0187],
        [-0.0849, -0.2169, -0.2106,  ..., -0.1089, -0.3100, -0.0206],
        [-0.0862, -0.2164, -0.2126,  ..., -0.1060, -0.3137, -0.0219]],
       device='cuda:0', grad_fn=<AddmmBackward>)


tensor(-4.3488, device='cuda:0', grad_fn=<SumBackward0>)

In [8]:
spacing = torch.rand(batch_size, 1).cuda()

m4 = Converter(copy.deepcopy(m1), copy.deepcopy(alignshift_conv_cfg), additional_forward_fts=['thickness'])
m4.eval()
m4.cuda()

ResNet(
  (conv1): AlignShiftConv(1, 64, kernel_size=(7, 7), stride=[1, 2, 2], padding=[0, 3, 3], dilation=[1, 1, 1], output_padding=[0, 0, 0], bias=False)
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): AlignShiftConv(64, 64, kernel_size=(3, 3), stride=[1, 1, 1], padding=[0, 1, 1], dilation=[1, 1, 1], output_padding=[0, 0, 0], bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): AlignShiftConv(64, 64, kernel_size=(3, 3), stride=[1, 1, 1], padding=[0, 1, 1], dilation=[1, 1, 1], output_padding=[0, 0, 0], bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): AlignShiftConv(64, 64, ker

In [9]:

out = m4(input, spacing)
print(out)

tensor([[-0.0363, -0.1440, -0.2694,  ..., -0.1056, -0.2693,  0.0502],
        [ 0.0008, -0.1672, -0.3468,  ..., -0.1464, -0.3323,  0.0956],
        [-0.0058, -0.1620, -0.3317,  ..., -0.1378, -0.3195,  0.0914],
        [-0.0130, -0.1527, -0.3108,  ..., -0.1243, -0.3019,  0.0738]],
       device='cuda:0', grad_fn=<AddmmBackward>)


In [10]:
out = m4(input, thickness=spacing)
print(out)

tensor([[-0.0363, -0.1440, -0.2694,  ..., -0.1056, -0.2693,  0.0502],
        [ 0.0008, -0.1672, -0.3468,  ..., -0.1464, -0.3323,  0.0956],
        [-0.0058, -0.1620, -0.3317,  ..., -0.1378, -0.3195,  0.0914],
        [-0.0130, -0.1527, -0.3108,  ..., -0.1243, -0.3019,  0.0738]],
       device='cuda:0', grad_fn=<AddmmBackward>)


In [12]:
sd = m4.state_dict()

In [29]:
m4.conv1.kernel_size

(7, 7)

In [18]:
m4.__getattr__('conv1.weight')

AttributeError: 'ResNet' object has no attribute 'conv1.weight'

In [14]:
sd['conv1.weight']

tensor([[[[-0.0404,  0.0473,  0.0016,  ...,  0.0367, -0.0227, -0.0355],
          [-0.0264, -0.0303,  0.0024,  ..., -0.0383,  0.0219,  0.0198],
          [-0.0492, -0.0025,  0.0398,  ...,  0.0131,  0.0475, -0.0184],
          ...,
          [-0.0444,  0.0055, -0.0133,  ..., -0.0088,  0.0362, -0.0134],
          [ 0.0254,  0.0054, -0.0235,  ...,  0.0182, -0.0173,  0.0102],
          [ 0.0183, -0.0741, -0.0029,  ...,  0.0051, -0.0043,  0.0095]]],


        [[[-0.0021, -0.0096,  0.0355,  ..., -0.0039,  0.0142,  0.0088],
          [ 0.0010,  0.0106,  0.0072,  ..., -0.0056,  0.0656, -0.0193],
          [ 0.0168,  0.0283,  0.0104,  ...,  0.0024, -0.0341, -0.0285],
          ...,
          [ 0.0035, -0.0136,  0.0351,  ...,  0.0073, -0.0165, -0.0077],
          [ 0.0480,  0.0351, -0.0217,  ...,  0.0008, -0.0273,  0.0149],
          [ 0.0019, -0.0247,  0.0025,  ...,  0.0202,  0.0252, -0.0080]]],


        [[[ 0.0172, -0.0365, -0.0414,  ...,  0.0046, -0.0662, -0.0222],
          [ 0.0029,  0.049

In [30]:
a = {'a':'a', 'b':'b'}

In [32]:
a.get('a')


'a'

In [6]:
m3 = nn.DataParallel(m2)

In [8]:
a.get('a')

NameError: name 'a' is not defined

In [6]:
m1

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
m1.eval()(input).sum()

RuntimeError: Expected 4-dimensional input for 4-dimensional weight 64 3 7 7, but got 5-dimensional input of size [4, 1, 7, 320, 320] instead

In [12]:
m4 = nn.DataParallel(m1)
m4.eval()(input).sum()

RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/hy/anaconda3/envs/torch-env/lib/python3.7/site-packages/torch/nn/parallel/parallel_apply.py", line 60, in _worker
    output = module(*input, **kwargs)
  File "/home/hy/anaconda3/envs/torch-env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 541, in __call__
    result = self.forward(*input, **kwargs)
  File "/home/hy/anaconda3/envs/torch-env/lib/python3.7/site-packages/torchvision/models/resnet.py", line 155, in forward
    x = self.layer1(x)
  File "/home/hy/anaconda3/envs/torch-env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 541, in __call__
    result = self.forward(*input, **kwargs)
  File "/home/hy/anaconda3/envs/torch-env/lib/python3.7/site-packages/torch/nn/modules/container.py", line 92, in forward
    input = module(input)
  File "/home/hy/anaconda3/envs/torch-env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 541

In [11]:
m4

NameError: name 'm4' is not defined